<a href="https://colab.research.google.com/github/szkjiro/program/blob/main/PyIris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データの自動分類

今回の話題は教師なし学習の一種（分類器の作成および実行表示）に属する問題です．
多変量解析ではクラスター分析と呼ばれてきました．

何をするかというと
1. データを分類する距離を定める
2. 距離の近いもの同士をそれぞれいくつかのグループに分ける
ということをします．

ここで扱うデータは，入門者用の例題としてよく使われる「フィッシャーのアヤメ」（IRISデータセット）です．
まずはデータの読み込みをします．


In [ ]:
# 必要なライブラリをインポート
# 距離計算のライブラリ
import numpy as np
# グラフィック表示ライブラリ
from matplotlib import pyplot as plt

# irisデータセットの使用
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data # 特徴量の行列, by sklearn

iris

最初に4つの数字が［］で挟まれて並びます．
これらはアヤメの3品種それぞれ50個体分のデータです．
属性に関する情報が数値の直後にいくつかあります．
target_namesという名前に続いてsetosaなど3つの名前，これはアヤメの品種にあたる名前です．
続いてアヤメの花びら（分類学上は萼片）のサイズとして，sepalおよびpetalの長さlengthおよび幅widthがありましたが，情報が多すぎましたね．
今後の処理で気にするところを改めて表示させます．


In [ ]:
# 品種の名前
print("Target names:", iris['target_names'])
# データの属性の名前
print("Freature names:", iris['feature_names'])
# headコマンドと同じ働きをするよう最初の6行を指定
print("head of data", iris['data'][:6])

このままでは，今まで扱ってきたデータフレーム（表計算でいえば，先頭行が各列のラベルであり，各列の2行目以降にデータがあるもの）ではないので，それをデータフレームに変換します．

In [ ]:
import pandas as pd

# データフレームに変換
Firis = pd.DataFrame(iris.data, columns=iris.feature_names)
Firis['species'] = iris.target
Firis.loc[Firis['species'] == 0, 'species'] = "setosa"
Firis.loc[Firis['species'] == 1, 'species'] = "versicolor"
Firis.loc[Firis['species'] == 2, 'species'] = "virginica"

# データフレーム変換後の確認
Firis.describe

どのようなデータか確認できました．これから機械学習による分類を行うわけですが，直感的に何が分類されるのかを，散布図行列により確認しておきましょう．見た目がよいのでseabornライブライで表示させます．

アヤメの属性（petal or sepal, length or widthなど）に応じて，３つの品種が混じって分布したり，分離したりが予想できることを観察しておいてください．


In [ ]:
# データ視覚化に向いたライブラリを使う
import seaborn as sns

sns.pairplot(Firis, hue='species')

分類し，それをデンドログラム（樹形図の一種）に表示します．
参考[階層的クラスタリング（S-Analysis）](https://data-analysis-stats.jp/機械学習/scikit-learnを用いた階層的クラスタリング-hierarchical-clusteringの解説/)

まずは実行してみてください．


## デンドログラムを通じて分類を見る

以下では[S-Analysisの階層的クラスタリングの記事](https://data-analysis-stats.jp/%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92/scikit-learn%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E9%9A%8E%E5%B1%A4%E7%9A%84%E3%82%AF%E3%83%A9%E3%82%B9%E3%82%BF%E3%83%AA%E3%83%B3%E3%82%B0-hierarchical-clustering%E3%81%AE%E8%A7%A3%E8%AA%AC/)を参考にしています．

irisデータセットの読み込みからやり直しています．
shape関数は，データの属性（petalおよびsepalにそれぞれlengthとwidhtがる）が4つあり，3品種全体で150データあることを表示します．


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import load_iris
from sklearn.cluster import AgglomerativeClustering

iris = load_iris()
# iris
X = iris.data
X.shape

デンドログラムを描く関数を先に用意しておきます．
このコードブロックは，関数定義のみなので，実行結果はありません．
しかし，このブロックを先に実行させる（関数を定義する）をしておかないと，以降の描画ができなくなります．
あとで，この関数に与えるレベルを変化させては再実行することで，デンドログラム全体の様子を詳しく観察します．

In [12]:
def plot_dendrogram(model, **kwargs):
   # 連結行列（linkage matrix）の計算およびデンドログラムの描画

   # 各ノード以下の葉の個数を数える
   counts = np.zeros(model.children_.shape[0])
   n_samples = len(model.labels_)
   for i, merge in enumerate(model.children_):
      current_count = 0
      for child_idx in merge:
         if child_idx < n_samples:
            current_count += 1 # 葉（末端）のノード
         else:
            current_count += counts[child_idx - n_samples]
      counts[i] = current_count

   linkage_matrix = np.column_stack([model.children_, model.distances_,
counts]).astype(float)

   # デンドログラムの描画
   dendrogram(linkage_matrix, **kwargs)

以下は標準的方法とされるユークリッド距離を用いたウォード法の実行プログラムです．
レベル設定のところを何通りかコメント行で準備しておきました．
レベルを変えて実行してみてください．

In [ ]:
# ユークリッド距離でウォード法を設定
model = AgglomerativeClustering(affinity='euclidean',
linkage='ward',
distance_threshold=0,
n_clusters=None)

model = model.fit(X)

plt.title('ward method with Euclidean metric')
# ３レベルまで描画
# ｐに与える値を書き換えて再実行してみよう
# plot_dendrogram(model, truncate_mode='level', p=1)
# plot_dendrogram(model, truncate_mode='level', p=5)
plot_dendrogram(model, truncate_mode='level', p=3)
plt.xlabel("number of data points under each node")
plt.show()

以下はマンハッタン距離を用いた平均法の実行プログラムです．上の例と同様に，レベルを変えて実行してみてください．



In [ ]:
model = AgglomerativeClustering(affinity='manhattan',
linkage='average',
distance_threshold=0,
n_clusters=None)

model = model.fit(X)

plt.title('Manhattan, linkage=average')
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode='level', p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

以下はコサイン距離（ベクトルの内積，相関係数に同じ）を用いた平均法の実行プログラムです．上の例と同様に，レベルを変えて実行してみてください．

In [ ]:
model = AgglomerativeClustering(affinity='cosine',
linkage='average',
distance_threshold=0,
n_clusters=None)

model = model.fit(X)

plt.title('Cosine, linkage=average')
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode='level', p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()